In [ ]:
# !pip install catboost

In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('dota_big_catboos.csv')

In [ ]:
players = ['player1_team_1', 'player2_team_1', 'player3_team_1',
           'player4_team_1', 'player5_team_1', 'player1_team_2',
           'player2_team_2', 'player3_team_2', 'player4_team_2',
           'player5_team_2']

heroes = ['hero1_team_1', 'hero2_team_1', 'hero3_team_1',
          'hero4_team_1', 'hero5_team_1', 'hero1_team_2',
          'hero2_team_2', 'hero3_team_2', 'hero4_team_2',
          'hero5_team_2']

In [ ]:
X, y = df.drop(['won_team_1'], axis=1), df['won_team_1']
X['data'] = (pd.to_datetime(X['data']) - pd.Timestamp("2018-01-01")) // pd.Timedelta('1d')

In [ ]:
un = []
for col in players:
    un += X[col].unique().tolist()

un = np.unique(un)

enc = OneHotEncoder()
enc.fit(un.reshape(-1, 1))

In [ ]:
players1_cols = ['player1_team_1', 'player2_team_1', 'player3_team_1', 'player4_team_1',
                 'player5_team_1']

playersteam1 = np.zeros((X.shape[0], len(un)))
for player in players1_cols:
    playersteam1 += enc.transform(X[player].to_numpy().reshape(-1, 1)).toarray()


players2_cols = ['player1_team_2', 'player2_team_2', 'player3_team_2',
                  'player4_team_2', 'player5_team_2']

playersteam2 = np.zeros((X.shape[0], len(un)))
for player in players2_cols:
    playersteam2 += enc.transform(X[player].to_numpy().reshape(-1, 1)).toarray()

In [ ]:
pca = PCA(n_components=50)
playersteam1_tranformed = pca.fit_transform(playersteam1)
playersteam2_tranformed = pca.transform(playersteam2)

In [ ]:
un = []
for col in heroes:
    un += X[col].unique().tolist()

un = np.unique(un)

enc = OneHotEncoder()
enc.fit(un.reshape(-1, 1))

In [ ]:
heroes1_cols = ['hero1_team_1', 'hero2_team_1', 'hero3_team_1',
                'hero4_team_1', 'hero5_team_1']

heroesteam1 = np.zeros((X.shape[0], len(un)))
for hero in heroes1_cols:
    heroesteam1 += enc.transform(X[hero].to_numpy().reshape(-1, 1)).toarray()

heroes2_cols = ['hero1_team_2',
                'hero2_team_2', 'hero3_team_2', 'hero4_team_2',
                'hero5_team_2']

heroesteam2 = np.zeros((X.shape[0], len(un)))
for hero in heroes2_cols:
    heroesteam2 += enc.transform(X[hero].to_numpy().reshape(-1, 1)).toarray()

In [ ]:
pca = PCA(n_components=50)
heroesteam1_tranformed = pca.fit_transform(heroesteam1)
heroesteam2_tranformed = pca.transform(heroesteam2)

In [ ]:
X_new = np.concatenate((X[['data', 'name_team_1', 'rank_team_1', 'ratio_team_1', 'points_team_1',
                           'prize_team_1', 'name_team_2', 'rank_team_2', 'ratio_team_2', 'points_team_2',
                           'prize_team_2']].to_numpy(),
                           playersteam1_tranformed, playersteam2_tranformed, heroesteam1_tranformed, heroesteam2_tranformed), axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_new, y, test_size=0.07, random_state=42, shuffle=False)

In [ ]:
cat_features = [1, 6]

In [ ]:
train_data = Pool(data=X_train,
                  label=y_train,
                  cat_features=cat_features)

test_data = Pool(data=X_test,
                label=y_test,
                cat_features=cat_features)

In [ ]:
model = CatBoostClassifier()
model.fit(train_data)
preds_class = model.predict(test_data)

Learning rate set to 0.064268
0:	learn: 0.6892250	total: 362ms	remaining: 6m 1s
1:	learn: 0.6853561	total: 536ms	remaining: 4m 27s
2:	learn: 0.6821946	total: 743ms	remaining: 4m 7s
3:	learn: 0.6793323	total: 945ms	remaining: 3m 55s
4:	learn: 0.6766941	total: 1.13s	remaining: 3m 45s
5:	learn: 0.6743188	total: 1.31s	remaining: 3m 36s
6:	learn: 0.6718230	total: 1.5s	remaining: 3m 33s
7:	learn: 0.6696600	total: 1.7s	remaining: 3m 31s
8:	learn: 0.6676383	total: 1.88s	remaining: 3m 26s
9:	learn: 0.6657659	total: 2.1s	remaining: 3m 27s
10:	learn: 0.6640487	total: 2.25s	remaining: 3m 22s
11:	learn: 0.6624201	total: 2.46s	remaining: 3m 22s
12:	learn: 0.6610609	total: 2.66s	remaining: 3m 21s
13:	learn: 0.6596859	total: 2.87s	remaining: 3m 22s
14:	learn: 0.6584811	total: 3.05s	remaining: 3m 20s
15:	learn: 0.6571564	total: 3.21s	remaining: 3m 17s
16:	learn: 0.6560564	total: 3.39s	remaining: 3m 15s
17:	learn: 0.6551064	total: 3.59s	remaining: 3m 15s
18:	learn: 0.6542661	total: 3.79s	remaining: 3m 1